<h2 style="text-align: center;" markdown="1">Journées Mathrice</h2>
<center>
    ![](images/logo_mathrice_small.png)
</center>

<h3 style="text-align: center;" markdown="1">Montpellier, 27-29 mars 2018</h3>
***
<h1 style="text-align: center;" markdown="0">Publier sur `math.cnrs.fr`</h1>
<h1 style="text-align: center;" markdown="0">avec GitLab Pages</h1>
***
<h4 style="text-align: center;" markdown="0">*Matthieu Boileau*</h4>
***

<center>
<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png"/>
</center>

![](images/stacked_wm_no_bg_small.png)

# GitLab Pages

À partir de projets hébergés sur <https://plmlab.math.cnrs.fr>, GitLab Pages permet de :

- publier un **site web statique**, construit par exemple avec un générateur de site statique,
- mettre en ligne des **pdf** d'articles de livres, de cours, d'énoncé d'exercices, etc., construits à partir de sources Tex, par exemple.
- mettre en ligne des **diaporamas**, comme celui-ci...



![](images/pages_group_avatar.png)

# Du point de vue l'utilisateur

## Permet de publier sur une URL en `math.cnrs.fr`

Pages permet de publier sur une URL qui contient un nom de projet ou d'équipe.

Pour le projet GitLab `projectname` qui appartient à
`username` ou `groupname`, l'adresse de publication sera :

```
https://[username|groupname].pages.math.cnrs.fr/[projectname]
```

> `projectname` est absent de l'URL s'il vaut `[username|groupname].pages.math.cnrs.fr`

## Permet de faire de l'édition collaborative

- Utiliser Git et GitLab pour **enregistrer et fusionner** les différentes contributions
- Utiliser GitLab Pages pour **reconstruire** le site ou le document en ligne après chaque modification sur le dépôt GitLab

## Très facile à mettre en oeuvre

En deux temps, trois mouvements :

- On enregistre un `gitlab runner` ou on utilise un *runner* partagé
- On ajoute un fichier `.gitlab-ci.yml` décrivant le job d'intégration continue

# Limitations

- on ne peut pas héberger de contenu web dynamique
- une taille maximale de contenu publiable (100Mo par défaut, peut être étendu par l'admin)
- l'URL de publication contient forcément `.pages.math.cnrs.fr` (si l'admin l'active, on peut [ajouter un nom de domaine personnalisé](https://docs.gitlab.com/ee/user/project/pages/introduction.html#add-a-custom-domain-to-your-pages-website))

![](images/pages_group_avatar.png)

# Du point de vue de l'administrateur

## Avantages

- aucune intervention nécessaire de la part de l'admin : la publication est automatiquement déclenchée par le job d'intégration continue
- on évite de gérer à la main "un site web => un serveur web (ou un virtualhost)" : Pages le fait automatiquement !
- pas de problème de sécurité lié à du contenu dynamique : l'exécution de code (php ou autre) n'est pas autorisée.

![](images/gear_small.png)

## Configuration

Quelques lignes dans le fichier `/etc/gitlab/gitlab.rb` :

```ruby
################
# gitlab pages #
################
pages_external_url 'https://pages.math.cnrs.fr'

pages_nginx['redirect_http_to_https'] = true
pages_nginx['ssl_certificate'] = "/etc/gitlab/ssl/star_pages_math_cnrs_fr.pem"
pages_nginx['ssl_certificate_key'] = "/etc/gitlab/ssl/star_pages_math_cnrs_fr.key"
```

où `star_pages_math_cnrs_fr.[pem|key]` sont les fichiers du certificat wildcard qui est nécessaire pour publier sur `*.pages.math.cnrs.fr`.

## Précautions

- il est très facile passer du contenu privé (projet gitlab) à du contenu public (Pages)
- ne publier que la branche master
- s'assurer que les utilisateurs ont adhéré à une charte

# Démonstration

On enregistre un *gitlab runner*

In [3]:
export CI_SERVER_URL="https://plmlab.math.cnrs.fr"
export RUNNER_NAME=mon_runner
export REGISTRATION_TOKEN=bhS4Qt87e-eEfaBthfmD  # Remplacez par la valeur indiquée sur la page de paramètres du runner
export REGISTER_NON_INTERACTIVE=true  # nécessaire pour le mode script

On installe

In [5]:
gitlab-runner register --executor docker --docker-image boileaum/jupyter

                                                   
Registering runner... succeeded                     runner=bhS4Qt87
Runner registered successfully. Feel free to start it, but if it's running already the config should be automatically reloaded! 
